# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
import os
import json
import holoviews
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,6.81,65,20,7.20,AR,1730339577
1,1,port-aux-francais,-49.3500,70.2167,4.22,84,25,10.88,TF,1730339578
2,2,arawa,-6.2132,155.5367,28.83,79,100,1.20,PG,1730339579
3,3,college,64.8569,-147.8028,-10.01,75,20,1.54,US,1730339580
4,4,sebastopol,44.6000,33.5333,11.11,63,15,4.09,UA,1730339582


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)
# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [33]:
### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_df = city_data_df[(city_data_df['Max Temp']>20)]

# Drop any rows with null values
cities_ideal = cities_narrow_df.dropna()

# Display sample data
cities_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,arawa,-6.2132,155.5367,28.83,79,100,1.20,PG,1730339579
6,6,ribeira grande,38.5167,-28.7000,20.99,89,9,7.15,PT,1730339584
9,9,cannonvale,-20.2833,148.7000,26.66,56,0,4.24,AU,1730339587
13,13,hamilton,39.1834,-84.5333,21.00,66,0,4.63,US,1730339589
16,16,san luis de la loma,17.2706,-100.8939,26.82,86,100,2.37,MX,1730339179
...,...,...,...,...,...,...,...,...,...,...
594,594,mahibadhoo,3.7833,72.9667,27.86,75,52,3.18,MV,1730340286
595,595,anna regina,7.2644,-58.5077,27.10,76,12,3.97,GY,1730340288
597,597,blackwater,-23.5833,148.8833,31.50,28,7,4.49,AU,1730340290
601,601,aoulef,26.9667,1.0833,25.20,23,74,4.35,DZ,1730340296


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=cities_ideal[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,arawa,PG,-6.2132,155.5367,79,
6,ribeira grande,PT,38.5167,-28.7000,89,
9,cannonvale,AU,-20.2833,148.7000,56,
13,hamilton,US,39.1834,-84.5333,66,
16,san luis de la loma,MX,17.2706,-100.8939,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
api_key = geoapify_key
latitude = 38.5167
longitude = -28.7000	
radius = 20000
params = {
    "api_key": api_key,
    "limit": 1,
    "filter": f"circle:{longitude},{latitude},{radius}",
    "bias": f"proximity:{longitude},{latitude}",
    "categories": "accommodation.hotel"
}

# Base URL for the Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

# Make the API request
response = requests.get(base_url, params=params)

# Convert the response to JSON format
name_address = response.json()

# Print the output
print(json.dumps(name_address, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Hospedaria JSF",
                "country": "Portugal",
                "country_code": "pt",
                "city": "Horta",
                "postcode": "9900-038",
                "district": "Horta (Ang\u00fastias)",
                "suburb": "Matriz",
                "street": "Rua Antiga Canada do Porto Pim",
                "lon": -28.632987399999998,
                "lat": 38.5301319,
                "formatted": "Hospedaria JSF, Rua Antiga Canada do Porto Pim, 9900-038 Horta, Portugal",
                "address_line1": "Hospedaria JSF",
                "address_line2": "Rua Antiga Canada do Porto Pim, 9900-038 Horta, Portugal",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "building",
                    "building.accommodation"
                ],


In [98]:
# Set parameters to search for a hotel
radius = 10000

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params = {
        "api_key": api_key,
        "limit": 20,
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "categories": "accommodation.hotel"
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        if name_address["features"]:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")

# Display sample data
hotel_df.head()

Starting hotel search
arawa - nearest hotel: Bovotel
ribeira grande - nearest hotel: Hospedaria JSF
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
hamilton - nearest hotel: North Vista Manor
san luis de la loma - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
trairi - nearest hotel: Pousada Villa Aurora
kavaratti - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
ceeldheer - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
howrah - nearest hotel: Sun India Guest House
alamos - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
margaret river - nearest hotel: Margaret River Hotel
hith

,City,Country,Lat,Lng,Humidity,Hotel Name
city_ID,,,,,,
2,arawa,PG,-6.2132,155.5367,79.0,Bovotel
6,ribeira grande,PT,38.5167,-28.7000,89.0,Hospedaria JSF
9,cannonvale,AU,-20.2833,148.7000,56.0,Whitsundays Rainforest Retreat
13,hamilton,US,39.1834,-84.5333,66.0,North Vista Manor
16,san luis de la loma,MX,17.2706,-100.8939,86.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_map= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size ="lat",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel name", "Country"]
)
    

# Display the map
hotel_map